<center>
<h1 style="font-family:verdana">
 🏘 Sistema de diàleg basat en regles 🏘 <h1>








<p>   🎯 <b>Objectiu</b>: en aquesta pràctica aprendrem a crear un assistent senzill de compra d'habitatges. El sistema haurà d'identificar i mostrar les cases que coincideixen amb les preferències de l'usuari. A poc a poc anirem afegint-hi funcionalitats, perquè el sistema siga més complet. </p>

<p> ✨ <b>Contingut</b>: en primer lloc, començarem amb un exemple senzill en què el sistema llançarà unes preguntes i respostes predefinides i l'usuari haurà d'escollir. A poc a poc anirem afegint-hi funcionalitats, perquè el sistema siga més complet. </p>

✏ <b>Exercicis</b>: en cada secció anireu trobant exercicis que haureu d'anar resolent.

---

<h2> Índex </h2>


1. [Fitxer JSON](#section-one)
  * [Exercici 1](#ex-one)
2. [Sistema de diàleg senzill](#section-two)
  * [Exercici 2](#ex-two)
  * [Exercici 3](#ex-three)
  * [Exercici 4](#ex-four)
3. [Millorem el sistema de diàleg](#section-three)
  * [Exercici 5](#ex-five)
  * [Exercici 6](#ex-six)
4. [Lliurable](#section-four)
---


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from nltk.tokenize.treebank import TreebankWordTokenizer
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

import json
import sys

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<h1><a name="section-one"> 1. Fitxer JSON </a></h1>

En primer lloc, carregarem el fitxer JSON per analitzar-ne l'estructura.

In [3]:
with open('/content/house_data.json') as f:
  data = json.load(f)

print('Start message: ', data['start_message'])
print('End message: ', data['end_message'])
print('Questions: ', data['questions'])
print('House: ', data['houses'])

Start message:  Welcome to the House Buying Assistant!
End message:  Thank you for using the House Buying Assistant. Goodbye!
Questions:  [{'question': 'How many bedrooms do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (1 - 5 bedrooms): ', 'answer_key': 'bedrooms'}, {'question': 'How many bathrooms do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (1 - 3 bathrooms): ', 'answer_key': 'bathrooms'}, {'question': 'What is your budget for the house?', 'type': 'numerical', 'prompt': 'Enter your choice (1k - 999k euros):', 'answer_key': 'price'}, {'question': 'How many square meters do you need?', 'type': 'numerical', 'prompt': 'Enter your choice (30 - 200 meters)', 'answer_key': 'square_meters'}, {'question': 'Which city or neighborhood would you prefer?', 'type': 'multichoice', 'prompt': 'Enter your choice', 'answer_key': 'location'}]
House:  [{'id': 1, 'type': 'rent', 'bedrooms': '3', 'bathrooms': '2', 'price': '1200', 'square_meters': '100', 'floor': '3', '

Podeu veure com al fitxer JSON podem trobar el missatge de benvinguda i el missatge de comiat. Les diferents preguntes predefinides que el sistema demanarà a l'usuari. I les cases que l'agència té disponibles per a oferir a l'usuari.

Si no acabeu d'entendre el format, podeu copiar el contingut del fitxer JSON [aquí](https://jsonviewer.stack.hu/) per visualitzar millor l'estructura.

---



 <h1><a name="ex-one"><center> ✏ Exercici 1 ✏</a></h1>


En aquest primer exercici us demanem que afegiu una nova casa manualment al fitxer JSON amb l'id 26. La resta de camps els podeu emplenar com vulgueu.

Si ho heu fet correctament podreu veure al següent *print* les dades que heu introduït.

In [4]:
house_26 = {'id': 26,
  'type': 'rent',
  'bedrooms': '3',
  'bathrooms': '2',
  'price': '1300',
  'square_meters': '90',
  'floor': '2',
  'elevator': 'Yes',
  'commercial_use': 'No',
  'terrace': 'Yes',
  'location': 'Barcelona'}
data['houses'].append(house_26)

In [ ]:
id = 26
house = data['houses'][id-1]

print(f"House ID: {house['id']}")
print(f"Bedrooms: {house['bedrooms']}")
print(f"Bathrooms: {house['bathrooms']}")
print(f"Price: {house['price']}")
print(f"Square Meters: {house['square_meters']}")
print(f"Location: {house['location']}")

House ID: 26
Bedrooms: 3
Bathrooms: 2
Price: 1300
Square Meters: 90
Location: Barcelona


---

<h1><a name="section-two"> 2. Sistema de diàleg senzill </a></h1>


En aquesta primera part crearem un sistema senzill que mostrarà les preguntes predefinides i, a continuació l'usuari haurà d'introduir la resposta. Tractarem de manera diferent les preguntes en què la resposta siga un número i les preguntes d'opció múltiple.

Intenteu entendre el funcionament, ja que treballarem sobre aquest codi.

In [5]:
def print_question(prompt, possible_options = []):
    print(prompt)
    if not len(possible_options) == 0:
      print("Options:", ", ".join(possible_options))

def initialize_available_options(house_data, available_options):
    for house in house_data['houses']:
        for key, value in house.items():
            available_options.setdefault(key, set()).add(value)

def preprocess_answer(answer):
    answer = nltk.word_tokenize(answer)
    return answer

def get_numerical_value(tok_answer):
    for token in tok_answer:
        if token.isnumeric() or token[:-1].isnumeric():
            return token
    return ''

#Exemple de frase tokenitzada
print(preprocess_answer('Vull un pis de 35k €'))

['Vull', 'un', 'pis', 'de', '35k', '€']


In [6]:
def process_numerical_question(question):
    print_question(question['question'])
    while True:
      answer = input(question['prompt'])
      tok_answer = preprocess_answer(answer)
      value = get_numerical_value(tok_answer)
      if not value == '':
        return value

def process_multichoice_question(question, options):
    print_question(question['question'], options)
    while True:
      answer = input(question['prompt'])
      if answer in options:
        return answer

user_preferences, available_options = {}, {}
initialize_available_options(data, available_options)

for question in data['questions']:
  answer_key = question['answer_key']
  possible_options = list(available_options.get(answer_key))

  if question['type'] == 'numerical':
    answer = process_numerical_question(question)
  else:
    answer = process_multichoice_question(question, possible_options)

  user_preferences[answer_key] = answer

How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 3
What is your budget for the house?
Enter your choice (1k - 999k euros):1k
How many square meters do you need?
Enter your choice (30 - 200 meters)30
Which city or neighborhood would you prefer?
Options: Esplugues de Llobregat, Barcelona, L'Hospitalet de Llobregat, Santa Coloma de Gramenet
Enter your choiceBarcelona




---


 <h1><a name="ex-two"><center> ✏ Exercici 2 ✏ </a></h1>

A continuació millorarem un poc el sistema. Us proposem que afegiu la frase de benvinguda que tenim guardada al fitxer JSON. És a dir, abans de mostrar la primera pregunta el sistema ens donarà la benvinguda.

In [7]:
# Creem una funció per executar el sistema de diàleg
def dialeg(data):
  user_preferences, available_options = {}, {}
  initialize_available_options(data, available_options)

  # Afegim la frase de benvinguda
  print(data['start_message'])

  for question in data['questions']:
    answer_key = question['answer_key']
    possible_options = list(available_options.get(answer_key))

    if question['type'] == 'numerical':
      answer = process_numerical_question(question)
    else:
      answer = process_multichoice_question(question, possible_options)

    user_preferences[answer_key] = answer

  return user_preferences

user_preferences = dialeg(data)

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 3
What is your budget for the house?
Enter your choice (1k - 999k euros):1k
How many square meters do you need?
Enter your choice (30 - 200 meters)30
Which city or neighborhood would you prefer?
Options: Esplugues de Llobregat, Barcelona, L'Hospitalet de Llobregat, Santa Coloma de Gramenet
Enter your choiceBarcelona


---


 <h1><a name="ex-three"><center> ✏ Exercici 3 ✏ </a></h1>

En aquest exercici us demanem que permeteu a l'usuari abandonar el programa quan ho desitge. Per exemple, si l'usuari escriu `quit`, que el sistema s'acomiade amb l'oració de comiat que trobareu al JSON i finalitze el programa.


🙃 `sys.exit()` genera una excepció SystemExit.

In [8]:
def process_numerical_question(question):
    print_question(question['question'])
    while True:
      answer = input(question['prompt'])
      #Afegim l'opció de que la resposta sigui "quit"
      if answer.lower() == "quit":
        return answer
      tok_answer = preprocess_answer(answer)
      value = get_numerical_value(tok_answer)
      if not value == '':
        return value

def process_multichoice_question(question, options):
    print_question(question['question'], options)
    while True:
      answer = input(question['prompt'])
      #Afegim l'opció de que la resposta sigui "quit"
      if answer in options or answer.lower() == "quit":
        return answer


def dialeg(data):
  user_preferences, available_options = {}, {}
  initialize_available_options(data, available_options)
  print(data['start_message'])

  for question in data['questions']:
    answer_key = question['answer_key']
    possible_options = list(available_options.get(answer_key))

    if question['type'] == 'numerical':
      answer = process_numerical_question(question)
    else:
      answer = process_multichoice_question(question, possible_options)

    # Afegim l'excepció SystemExit
    if answer.lower() == "quit":
      print(data['end_message'])
      sys.exit()

    user_preferences[answer_key] = answer

  return user_preferences

user_preferences = dialeg(data)

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): quit
Thank you for using the House Buying Assistant. Goodbye!


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Funciona també si escriviu `Quit`?

Sí, funciona perquè en la condició que comprova si s'ha d'aturar el sistema el text es passa a minúscules:  *answer.lower() == "quit"*



---

 <h1><a name="ex-four"><center> ✏ Exercici 4 ✏ </a></h1>

Fins ara tenim a `user_preferences` les opcions que l'usuari ha escollit.







Per tant, ara haurem de revisar si l'agència disposa d'alguna casa disponible amb aquestes característiques. Per això us proposem completar la funció `find_suitable_houses`. Aquesta funció tornarà una llista amb tants diccionaris com cases disponibles. Per exemple si segons les preferències de l'usuari només està disponible la casa amb id 3, `suitable_house` serà:


```
[{'id': 3, 'bedrooms': '2', 'bathrooms': '1', 'price': '160k', 'square_meters': '80', 'location': 'Santa Coloma de Gramenet'}]
```

In [9]:
def k_to_number(value):
  """
  Funció que converteix els preus en milers (10k) a enters sense text (10000)
  """
  if value[-1] == 'k':
    return int(value[:-1]*1000)
  return int(value)

def find_suitable_houses(data, user_preferences):
  """
  Funció que revisa les preferències de l'usuari i troba aquelles cases
  que s'ajusten a les preferències o tenen unes millors característiques.
  """
  suitable_houses = []
  for house in data['houses']:
    is_suitable = True
    for key in user_preferences:
      if key == 'location' and house[key] != user_preferences[key]:
        is_suitable = False
        break
      elif key in ['bedrooms', 'bathrooms', 'square_meters'] and (int(house[key]) < int(user_preferences[key])):
        is_suitable = False
        break
      elif key == 'price' and k_to_number(house[key]) > k_to_number(user_preferences[key]):
        is_suitable = False
        break
    if is_suitable:
      suitable_houses.append(house)
  return suitable_houses

---



Una vegada tingueu la funció `find_suitable_houses` programada, podreu executar el següent *script* que us mostrarà, si n'hi ha, la casa o cases disponibles amb les característiques que heu triat.

In [11]:
def print_suitable_houses(suitable_houses):
  if suitable_houses:
    print("\nBased on your preferences, the most suitable houses are:")
    for house in suitable_houses:
      print(f"House ID: {house['id']}")
      print(f"Type: {house['type']}")
      print("Bedrooms:", house['bedrooms'])
      print("Bathrooms:", house['bathrooms'])
      print("Price:", house['price'], "euros")
      print("Square Meters:", house['square_meters'], "m^2")
      print("Elevator:", house['elevator'])
      print("Location:", house['location'])
      print()
  else:
    print("\nSorry, no suitable houses match your preferences. \n")

user_preferences = dialeg(data)
suitable_houses = find_suitable_houses(data, user_preferences)
print_suitable_houses(suitable_houses)

Welcome to the House Buying Assistant!
How many bedrooms do you need?
Enter your choice (1 - 5 bedrooms): 1
How many bathrooms do you need?
Enter your choice (1 - 3 bathrooms): 1
What is your budget for the house?
Enter your choice (1k - 999k euros):999k
How many square meters do you need?
Enter your choice (30 - 200 meters)30
Which city or neighborhood would you prefer?
Options: Esplugues de Llobregat, Barcelona, L'Hospitalet de Llobregat, Santa Coloma de Gramenet
Enter your choiceBarcelona

Based on your preferences, the most suitable houses are:
House ID: 17
Type: rent
Bedrooms: 1
Bathrooms: 1
Price: 450 euros
Square Meters: 45 m^2
Elevator: No
Location: Barcelona

House ID: 18
Type: rent
Bedrooms: 1
Bathrooms: 1
Price: 850 euros
Square Meters: 50 m^2
Elevator: Yes
Location: Barcelona

House ID: 19
Type: rent
Bedrooms: 3
Bathrooms: 1
Price: 600 euros
Square Meters: 75 m^2
Elevator: No
Location: Barcelona

House ID: 20
Type: sale
Bedrooms: 3
Bathrooms: 2
Price: 360k euros
Square Mete

<h1><a name="section-three"> 3. Millorem el sistema de diàleg </a></h1>


---


 <h1><a name="ex-five"><center> ✏ Exercici 5 ✏ </a></h1>

Tal com haureu comprovat sempre esteu obligats a triar una opció. En aquest últim exercici proposem que afegiu l'opció de no triar-ne cap. És a dir, si ens és igual el nombre d'habitacions, podrem posar per exemple `any` i el sistema ens farà la següent pregunta.

In [12]:
def process_numerical_question(question):
    print_question(question['question'])
    while True:
      answer = input(question['prompt'])
      #Afegim l'opció de que la resposta sigui "any" o "quit"
      if answer.lower() == "quit" or answer.lower() == "any":
        return answer
      tok_answer = preprocess_answer(answer)
      value = get_numerical_value(tok_answer)
      if not value == '':
        return value

def process_multichoice_question(question, options):
    print_question(question['question'], options)
    while True:
      answer = input(question['prompt'])
      #Afegim l'opció de que la resposta sigui "any" o "quit"
      if answer in options or answer.lower() == "quit" or answer.lower() == "any":
        return answer

In [13]:
def find_suitable_houses(data, user_preferences):
  """
  Funció que revisa les preferències de l'usuari i troba aquelles cases
  que s'ajusten a les preferències o tenen unes millors característiques.
  """
  suitable_houses = []
  for house in data['houses']:
    is_suitable = True
    for key in user_preferences:
      # Afegim la comparació amb "any"
      if user_preferences[key] != "any":
          if key == 'location' and house[key] != user_preferences[key]:
            is_suitable = False
            break
          elif key in ['bedrooms', 'bathrooms', 'square_meters'] and (int(house[key]) < int(user_preferences[key])):
            is_suitable = False
            break
          elif key == 'price' and k_to_number(house[key]) > k_to_number(user_preferences[key]):
            is_suitable = False
            break
    if is_suitable:
      suitable_houses.append(house)
  return suitable_houses

---



 <h1><a name="ex-six"><center> ✏ Exercici 6 ✏ </a></h1>

Amplieu el vostre xatbot perquè tinga en compte la següent informació sobre:

1. **Tipus d'habitatge:** Pregunteu a l'usuari si vol una casa per a compra (tipus *"sale"* a les dades) o per llogar (tipus *"rent"*). Al final de l'execució, el sistema haurá de mostrar habitatges amb el tipus seleccionat.

2. **Ingressos:** S'aconsella no dedicar més del 35% dels ingressos a l'habitatge. En cas d'escollir lloguer, preguntar a l'usuari els ingressos mensuals de la seua llar, i només mostrar aquells que estiguen per sota del 35%.

3. **Planta:** Preguntar als usuaris a quina planta volen viure. Al final de l'execució, només s'oferiràn habitatges que estiguen en aquella planta o superior.

4. **Terrassa:** Preguntar a l'usuari si vol un habitatge amb terrassa o no. En cas de voler terrassa, només s'oferiran habitatges amb terrassa. En cas contrari, oferir habitatges amb terrassa o sense.

5. **Ascensor:** Preguntar a l'usuari si vol un habitatge amb ascensor o no. En cas de voler ascensor, només s'oferiran habitatges amb ascensor. En cas contrari, oferir habitatges amb ascensor o sense.

6. **Us comercial:** Preguntar a l'usuari si vol fer servir l'habitatge com a negoci. En cas afirmatiu, només oferir habitatges amb la propietat *"commercial use"*

---





In [ ]:
# Exercici resolt en la versió final

 <h1><a name="ex-seven"><center> ✏ Exercici 7 ✏ </a></h1>

Ara ja teniu un sistema funcional. Aplicant el mètode vist a classe (Exemples, Camins, Prototipat i Proves) milloreu aquest sistema per donar una millor experiència als usuaris.

In [ ]:
# Exercici resolt en la versió final

<h1><a name="section-four"> 4. Lliurable </a></h1>

Heu d'entregar un document PDF de com a **màxim 10 pàgines** que incloga els resultats de tots els exercicis així com una explicació de cadascun dels resultats i de la modificació que heu fet. L'estructura del document és:

1. Introducció.
2. Resultats (amb raonament).
3. Conversation flow del vostre sistema final.
4. Conclusions.

No cal que afegiu el vostre codi al document, podeu entregar el *notebook* juntament amb el document.


In [14]:
with open('/content/house_data.json') as f:
  data = json.load(f)

# MODIFICAR ELS PROMPTS PERQUÈ SIGUIN MÉS LLEGIBLES
for question in data['questions']:
    if question['answer_key'] == 'square_meters':
        question['prompt'] = 'Enter your choice (30 - 200 square meters): '
    elif question['answer_key'] == 'location':
        question['prompt'] = 'Enter your choice: '
    elif question['answer_key'] == 'price':
        question['prompt'] = 'Enter your choice (1k - 999k euros): '


# EXERCICI 6
typehouse = {'question': 'Do you want to buy or rent the house?', 'type': 'multichoice', 'prompt': 'Enter your choice: ', 'answer_key': 'type'}
data['questions'].append(typehouse)
floor = {"question": "In which floor do you want to live?", "type": "numerical", "prompt": "Enter your choice (0 - 8 floors): ", "answer_key": "floor"}
data['questions'].append(floor)
terrace = {"question": "Do you want terrace?", "type": "multichoice", "prompt": "Enter your choice: ", "answer_key": "terrace"}
data['questions'].append(terrace)
elevator = {'question': 'Do you need an elevator?', 'type': 'multichoice', 'prompt': 'Enter your choice: ', 'answer_key': 'elevator'}
data['questions'].append(elevator)
commercial_use = {'question': 'Do you want to give it a commercial use?', 'type': 'multichoice', 'prompt': 'Enter your choice: ', 'answer_key': 'commercial_use'}
data['questions'].append(commercial_use)


# TRACTEM ELS MISSATGES I N'AFEGIM DE NOUS
if isinstance(data.get('start_message'), str):
    data['start_message'] = [data.pop('start_message')]
if isinstance(data.get('end_message'), str):
    data['end_message'] = [data.pop('end_message')]

  # Start message
data['start_message'].append("Let's start your house hunting journey!😊")
data['start_message'].append("Hi there! Ready to find your next home?🔍")
data['start_message'].append("I'm here to help you find the perfect home for you.🏡")

  # End message
data['end_message'].append("Thank you for visiting! Good luck with your house search.👋")
data['end_message'].append("That's all from me! Feel free to come back anytime.😊")

  # Instructions
data['system_instructions'] = "\nWhile you're answering the questions, feel free to use these options anytime:\n    - QUIT: Just type 'quit' if you want to end the session. I'll stop right away.\n    - ANY: If you type 'any', I'll understand that you don't want to pick any of the options and we’ll move on to the next question.\n    - RETURN: Want to change your last answer? Type 'return' and I'll take you back to the previous question so you can fix it."

  # Frases de confirmació
data['confirmation_messages'] = []
data['confirmation_messages'].append("Got it! {preference} is set to {value}.")
data['confirmation_messages'].append("You chose {preference} as {value}.")
data['confirmation_messages'].append("Thanks! You've set {preference} as {value}.")


  # Respostes invàlides
data['invalid_responses'] = {
        "default": [
            "I didn't quite get that, can you try again?",
            "Hmm, that doesn't seem right. Please try again.",
            "Sorry, that wasn't an option. Could you repeat?",
            "I'm not sure what you meant. Could you rephrase that?"],
        "not_a_number": [
            "That doesn't look like a number. Could you enter a number?",
            "Please, I need a number here.",
            "I was expecting a number. Could you type one?",
            "Oops! Looks like you entered a word instead of a number. Please try again with a number.",
            "I need a valid number to continue. Can you try again?"],
        "out_of_range": [
            "That number is out of the expected range. Please enter a valid number.",
            "Oops! The number is too high or too low. Try a number within the range.",
            "Looks like the number is outside the limits. Please enter a number in the correct range.",
            "That doesn't seem to be a valid option. Please pick a number within the allowed range.",
            "The number is not in the expected range. Could you try again with a number within the limits?"]}

# AFEGIR VARIACIONS A LES PREGUNTES
for question in data['questions']:
    if question['answer_key'] == 'bedrooms':
        question['question'] = [
            'How many bedrooms do you need?',
            'How many bedrooms would be ideal for you?',
            'What number of bedrooms are you looking for?',
            'How many bedrooms would suit your needs?'
        ]

    elif question['answer_key'] == 'bathrooms':
        question['question'] = [
            'How many bathrooms do you need?',
            'How many bathrooms do you prefer?',
            'What number of bathrooms would work for you?',
            'How many bathrooms are necessary?'
        ]

    elif question['answer_key'] == 'price':
        question['question'] = [
            'What is your budget for the house?',
            'What’s the maximum budget you have in mind?',
            'How much are you willing to spend on the house?',
            'What’s your ideal price range for the house?'
        ]

    elif question['answer_key'] == 'square_meters':
        question['question'] = [
            'How many square meters do you need?',
            'What’s the minimum size you’re looking for (in m^2)?',
            'How big of a house do you want (in square meters)?',
            'How many square meters would be ideal for you?'
        ]

    elif question['answer_key'] == 'location':
        question['question'] = [
            'Which city or neighborhood would you prefer?',
            'In which city or neighborhood would you like to live?',
            'Where would you prefer to settle?',
            'What location do you have in mind for your new home?'
        ]

    elif question['answer_key'] == 'type':
        question['question'] = [
            'Do you want to buy or rent the house?',
            'Are you looking to rent or buy?',
            'Would you prefer to rent or purchase the property?',
            'Do you plan to buy or rent your home?'
        ]

    elif question['answer_key'] == 'floor':
        question['question'] = [
            'In which floor do you want to live?',
            'Which floor level would you like to live on?',
            'Do you prefer a specific floor in the building?',
            'What floor would you like your house to be on?'
        ]

    elif question['answer_key'] == 'terrace':
        question['question'] = [
            'Do you want a terrace?',
            'Would you like a house with a terrace?',
            'Are you interested in having a terrace?',
            'Do you prefer your house to include a terrace?'
        ]

    elif question['answer_key'] == 'elevator':
        question['question'] = [
            'Do you need an elevator?',
            'Is an elevator a must for you?',
            'Would you need an elevator in the building?',
            'Do you prefer having access to an elevator?'
        ]

    elif question['answer_key'] == 'commercial_use':
        question['question'] = [
            'Do you want to give it a commercial use?',
            'Would you use the property for commercial purposes?',
            'Are you planning to use it for business?',
            'Is commercial use something you have in mind?'
        ]

In [15]:
# PER DONAR MÉS OPCIONS AL CLIENT, AFEGIM MÉS CASES

house_26 = {'id': 26,'type': 'rent','bedrooms': '3','bathrooms': '2','price': '1300','square_meters': '90','floor': '2','elevator': 'Yes','commercial_use': 'No','terrace': 'Yes','location': 'Barcelona'}
house_27 = {'id': 27,'type': 'sale','bedrooms': '3','bathrooms': '1','price': '270k','square_meters': '85','floor': '3','elevator': 'No','commercial_use': 'No','terrace': 'Yes','location': "L'Hospitalet de Llobregat"}
house_28 = {'id': 28,'type': 'rent','bedrooms': '2','bathrooms': '1','price': '950','square_meters': '70','floor': '4','elevator': 'Yes','commercial_use': 'No','terrace': 'No','location': "Esplugues de Llobregat"}
house_29 = {'id': 29,'type': 'sale','bedrooms': '5','bathrooms': '3','price': '550k','square_meters': '160','floor': '2','elevator': 'Yes','commercial_use': 'Yes','terrace': 'Yes','location': "Barcelona"}
house_30 = {'id': 30,'type': 'rent','bedrooms': '4','bathrooms': '2','price': '1800','square_meters': '110','floor': '5','elevator': 'No','commercial_use': 'No','terrace': 'Yes','location': "L'Hospitalet de Llobregat"}
house_31 = {'id': 31,'type': 'sale','bedrooms': '2','bathrooms': '2','price': '210k','square_meters': '95','floor': '1','elevator': 'Yes','commercial_use': 'No','terrace': 'No','location': "Barcelona"}
house_32 = {'id': 32,'type': 'rent','bedrooms': '1','bathrooms': '1','price': '750','square_meters': '45','floor': '3','elevator': 'No','commercial_use': 'No','terrace': 'Yes','location': "Esplugues de Llobregat"}
house_33 = {'id': 33,'type': 'sale','bedrooms': '4','bathrooms': '3','price': '320k','square_meters': '140','floor': '4','elevator': 'Yes','commercial_use': 'Yes','terrace': 'Yes','location': "Santa Coloma de Gramenet"}
house_34 = {'id': 34,'type': 'rent','bedrooms': '2','bathrooms': '1','price': '1200','square_meters': '60','floor': '6','elevator': 'Yes','commercial_use': 'No','terrace': 'Yes','location': "Barcelona"}
house_35 = {'id': 35,'type': 'sale','bedrooms': '3','bathrooms': '2','price': '290k','square_meters': '100','floor': '5','elevator': 'Yes','commercial_use': 'No','terrace': 'Yes','location': "L'Hospitalet de Llobregat"}
house_36 = {'id': 36,'type': 'rent','bedrooms': '3','bathrooms': '2','price': '1600','square_meters': '90','floor': '2','elevator': 'No','commercial_use': 'Yes','terrace': 'No','location': "Barcelona"}
house_37 = {'id': 37,'type': 'sale','bedrooms': '5','bathrooms': '3','price': '500k','square_meters': '150','floor': '3','elevator': 'Yes','commercial_use': 'Yes','terrace': 'Yes','location': "Esplugues de Llobregat"}
house_38 = {'id': 38,'type': 'rent','bedrooms': '2','bathrooms': '2','price': '1100','square_meters': '85','floor': '4','elevator': 'No','commercial_use': 'No','terrace': 'Yes','location': "Santa Coloma de Gramenet"}
house_39 = {'id': 39,'type': 'sale','bedrooms': '3','bathrooms': '1','price': '260k','square_meters': '90','floor': '1','elevator': 'No','commercial_use': 'No','terrace': 'Yes','location': "Barcelona"}
house_40 = {'id': 40,'type': 'rent','bedrooms': '4','bathrooms': '2','price': '2200','square_meters': '120','floor': '7','elevator': 'Yes','commercial_use': 'No','terrace': 'Yes','location': "Esplugues de Llobregat"}

data['houses'].append(house_26)
data['houses'].append(house_27)
data['houses'].append(house_28)
data['houses'].append(house_29)
data['houses'].append(house_30)
data['houses'].append(house_31)
data['houses'].append(house_32)
data['houses'].append(house_33)
data['houses'].append(house_34)
data['houses'].append(house_35)
data['houses'].append(house_36)
data['houses'].append(house_37)
data['houses'].append(house_38)
data['houses'].append(house_39)
data['houses'].append(house_40)



In [16]:
print('Start message: ', data['start_message'])
print('End message: ', data['end_message'])
print('Questions: ', data['questions'])
print('House: ', data['houses'])
print('Confirmation messages: ', data['confirmation_messages'])
print('Invalid responses: ', data['invalid_responses'])

Start message:  ['Welcome to the House Buying Assistant!', "Let's start your house hunting journey!😊", 'Hi there! Ready to find your next home?🔍', "I'm here to help you find the perfect home for you.🏡"]
End message:  ['Thank you for using the House Buying Assistant. Goodbye!', 'Thank you for visiting! Good luck with your house search.👋', "That's all from me! Feel free to come back anytime.😊"]
Questions:  [{'question': ['How many bedrooms do you need?', 'How many bedrooms would be ideal for you?', 'What number of bedrooms are you looking for?', 'How many bedrooms would suit your needs?'], 'type': 'numerical', 'prompt': 'Enter your choice (1 - 5 bedrooms): ', 'answer_key': 'bedrooms'}, {'question': ['How many bathrooms do you need?', 'How many bathrooms do you prefer?', 'What number of bathrooms would work for you?', 'How many bathrooms are necessary?'], 'type': 'numerical', 'prompt': 'Enter your choice (1 - 3 bathrooms): ', 'answer_key': 'bathrooms'}, {'question': ['What is your budget 

In [17]:
import re
def k_to_number(value):
  """
  Funció que converteix els preus en milers (10k) a enters sense text (10000)
  """
  if str(value)[-1] != 'k':
    return int(value)
  else:
    return int(int(value[:-1])*1000)

def extract_min_max(prompt):
  # Trobar números amb o sense 'k' al final
  matches = re.findall(r'\d+k?|\d+', prompt)
  if len(matches) >= 2:
    min_val = matches[0]
    max_val = matches[1]
    return min_val, max_val
  return None, None

In [18]:
def print_question(prompt, possible_options = []):
    print(f'\n{prompt}')
    if not len(possible_options) == 0:
      print("Options:", ", ".join(possible_options))

def initialize_available_options(house_data, available_options):
    for house in house_data['houses']:
        for key, value in house.items():
            available_options.setdefault(key, set()).add(value)

def preprocess_answer(answer):
    answer = nltk.word_tokenize(answer)
    return answer

def get_numerical_value(tok_answer):
    for token in tok_answer:
        if token.isnumeric() or token[:-1].isnumeric():
            return token
    return ''

In [19]:
import random

def handle_invalid_response(response_type="default", data=None):
    return random.choice(data['invalid_responses'][response_type])

def get_random_start_message(data):
    return random.choice(data['start_message'])

def get_random_end_message(data):
    return random.choice(data['end_message'])

def inform_about_preference(preference, value, data):
    responses = data['confirmation_messages']
    return random.choice(responses).format(preference=preference, value=value)

In [20]:
def process_numerical_question(question):
    print_question(random.choice(question['question']))
    if question['answer_key'] != 'income':
      min_value, max_value = extract_min_max(question['prompt'])
    while True:
      answer = input(question['prompt'])
      if answer.lower() == "quit" or answer.lower() == "any" or answer.lower() == "return":
        return answer
      tok_answer = preprocess_answer(answer)
      value = get_numerical_value(tok_answer)

      if value == '':
        print(handle_invalid_response("not_a_number", data))
      elif question['answer_key'] == 'income' or k_to_number(min_value) <= k_to_number(value) <= k_to_number(max_value):
        return value
      else:
        print(handle_invalid_response("out_of_range", data))


def process_multichoice_question(question, options):
    print_question(random.choice(question['question']), options)
    while True:
      answer = input(question['prompt'])
      lower_options = [option.lower() for option in options]
      if answer.lower() in lower_options or answer.lower() == "quit" or answer.lower() == "any" or answer.lower() == "return":
        return answer
      print(handle_invalid_response("default", data))

In [21]:
def treat_income(user_preferences):

    # Guardar el preu original
    original_price = user_preferences['price']

    while True:
        answer2 = process_numerical_question({'question': ['What is your monthly income?', 'Could you tell me your monthly income?'], 'type': 'numerical', 'prompt': 'Enter your monthly income: ', 'answer_key': 'income'})
        answer2_lower = str(answer2).lower()

        # # Com que 'income' es tracta diferent, tornem a tractar els casos 'any', 'quit' i 'return'
        if answer2_lower == "quit":
            print(get_random_end_message(data))
            sys.exit()

        elif answer2_lower == "return":
            print("Returning to the previous question...")
            return False  # Indica que l'usuari vol retornar

        elif answer2_lower != "any":
            income_value = k_to_number(answer2) * 0.35
            print(inform_about_preference('income', answer2, data))
            if original_price == 'any':
                user_preferences['price'] = income_value
            else:
                user_preferences['price'] = min(k_to_number(original_price), income_value)
            return True  # Indica que l'income s'ha ajustat correctament

        else:
            print("Please, I need you to answer this question.")

In [22]:
def dialeg(data):
    """
    Funció que revisa les preferències de l'usuari i troba aquelles cases
    que s'ajusten a les preferències o tenen unes millors característiques.
    """
    user_preferences, available_options = {}, {}
    initialize_available_options(data, available_options)
    print(get_random_start_message(data))
    print(data["system_instructions"])

    rent = False
    i = 0
    while i < len(data['questions']):

        question = data['questions'][i]
        answer_key = question['answer_key']
        possible_options = list(available_options.get(answer_key, []))

        if question['type'] == 'numerical':
            answer = process_numerical_question(question)
        else:
            answer = process_multichoice_question(question, possible_options)

        answer_lower = str(answer).lower()

        if answer_lower == "quit":
            print(get_random_end_message(data))
            sys.exit()

        elif answer_lower == "return":
            if answer_key == 'floor' and rent:
                print("Returning to the previous question...")
                success = treat_income(user_preferences)
                if not success:
                    i -= 1  # Anem a la pregunta anterior
                continue
            elif i > 0:
                print("Returning to the previous question...")
                i -= 1
                continue
            else:
                print("There is no previous question.")
                continue

        else:
          print(inform_about_preference(answer_key, answer, data))

        if answer_key == 'type':
            if answer_lower in ['rent', 'any']:
                rent = True
                success = treat_income(user_preferences)
                if not success:
                    continue
            else:
                rent = False

        user_preferences[answer_key] = answer
        i += 1

    return user_preferences

In [23]:
def find_suitable_houses(data, user_preferences):
  """
  Funció que revisa les preferències de l'usuari i troba aquelles cases
  que s'ajusten a les preferències o tenen unes millors característiques.
  """
  suitable_houses = []
  for house in data['houses']:
    is_suitable = True
    for key in user_preferences:
      if user_preferences[key] != "any":
          if key in ['bedrooms', 'bathrooms', 'square_meters', 'floor'] and (int(house[key]) < int(user_preferences[key])):
            is_suitable = False
            break
          elif key == 'price' and k_to_number(house[key]) > k_to_number(user_preferences[key]):
            is_suitable = False
            break
          elif key in ['elevator', 'terrace', 'commercial_use'] and house[key].lower() == 'no' and user_preferences[key].lower() == 'yes':
            is_suitable = False
            break
          elif key in ['location', 'type'] and house[key].lower() != user_preferences[key].lower():
            is_suitable = False
            break
    if is_suitable:
      suitable_houses.append(house)
  return suitable_houses

In [24]:
def print_suitable_houses(suitable_houses):
  if suitable_houses:
    print("\nBased on your preferences, the most suitable houses are:")
    for house in suitable_houses:
      print(f"House ID: {house['id']}")
      print(f"Type: {house['type']}")
      print("Bedrooms:", house['bedrooms'])
      print("Bathrooms:", house['bathrooms'])
      print("Price:", house['price'], "euros")
      print("Square Meters:", house['square_meters'], "m^2")
      print("Floor:", house['floor'])
      print("Elevator:", house['elevator'])
      print("Commercial use:", house['commercial_use'])
      print("Terrace:", house['terrace'])
      print("Location:", house['location'])
      print()
  else:
    print("\nSorry, no suitable houses match your preferences. \n")

user_preferences = dialeg(data)
suitable_houses = find_suitable_houses(data, user_preferences)
print_suitable_houses(suitable_houses)

Hi there! Ready to find your next home?🔍

While you're answering the questions, feel free to use these options anytime:
    - QUIT: Just type 'quit' if you want to end the session. I'll stop right away.
    - ANY: If you type 'any', I'll understand that you don't want to pick any of the options and we’ll move on to the next question.
    - RETURN: Want to change your last answer? Type 'return' and I'll take you back to the previous question so you can fix it.

What number of bedrooms are you looking for?
Enter your choice (1 - 5 bedrooms): three
Oops! Looks like you entered a word instead of a number. Please try again with a number.
Enter your choice (1 - 5 bedrooms): 3
Got it! bedrooms is set to 3.

What number of bathrooms would work for you?
Enter your choice (1 - 3 bathrooms): return
Returning to the previous question...

What number of bedrooms are you looking for?
Enter your choice (1 - 5 bedrooms): 3
You chose bedrooms as 3.

How many bathrooms do you prefer?
Enter your choice (

---